# EPA Fuel Economy Data Assessment

This jupyter notebook provides an assessment of EPA fuel economy data from 2018 and 2008.

To download data: https://www.fueleconomy.gov/feg/download.shtml

Documentation: https://www.fueleconomy.gov/feg/EPAGreenGuide/GreenVehicleGuideDocumentation.pdf

README.txt: http://www.fueleconomy.gov/feg/epadata/Readme.txt

In [158]:
import pandas as pd

#import and assess the 2008 data
temp_08 = pd.read_excel("https://www.fueleconomy.gov/feg/EPAGreenGuide/xls/all_alpha_08.xls")
temp_18 = pd.read_excel('https://www.fueleconomy.gov/feg/EPAGreenGuide/xls/all_alpha_18.xlsx')

# REMOVE THE BELOW

In [159]:
df_08 = temp_08.copy()
df_18 = temp_18.copy()

In [160]:
#evaluate column labels to determine if they are aligned
print("2008", df_08.columns, "2018", df_18.columns, 
      "\nDo all columns match?",
      (df_08.columns == df_18.columns).all(),
      sep='\n')

2008
Index(['Model', 'Displ', 'Cyl', 'Trans', 'Drive', 'Fuel', 'Sales Area', 'Stnd',
       'Underhood ID', 'Veh Class', 'Air Pollution Score', 'FE Calc Appr',
       'City MPG', 'Hwy MPG', 'Cmb MPG', 'Unadj Cmb MPG',
       'Greenhouse Gas Score', 'SmartWay'],
      dtype='object')
2018
Index(['Model', 'Displ', 'Cyl', 'Trans', 'Drive', 'Fuel', 'Cert Region',
       'Stnd', 'Stnd Description', 'Underhood ID', 'Veh Class',
       'Air Pollution Score', 'City MPG', 'Hwy MPG', 'Cmb MPG',
       'Greenhouse Gas Score', 'SmartWay', 'Comb CO2'],
      dtype='object')

Do all columns match?
False


In [161]:
def clean_columns(df):

    #Make columns lowercase and replace spaces with underscores in column names
    df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)
    
    #remove null values and duplicates
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)

In [162]:
#dropping columns that are not present in both datasets or are unnecessary for evaluation
df_08.drop(['Stnd', 'Underhood ID', 'FE Calc Appr', 'Unadj Cmb MPG'],axis=1,inplace=True)
df_18.drop(['Stnd', 'Stnd Description', 'Underhood ID', 'Comb CO2'],axis=1,inplace=True)

#renaming columns for consistency in 2008 only

df_08.rename(columns={'Sales Area':'Cert Region'}, inplace=True)

clean_columns(df_08)
clean_columns(df_18)

Following columns do not have matching types: cyl, air_pollution_score, city_mpg, hwy_mpg, cmb_mpg, greenhouse_gas_score

In [163]:
#remove FC certifications because they are not present in the 2018 dataset
cert_08 = df_08.query('cert_region == "FC"')
df_08.drop(cert_08.index, inplace=True)

#2008 - convert from string to integer
#2018 - convert from integer to string
df_08['cyl'] = df_08['cyl'].apply(lambda x: int(x.strip('( cyl)')))
df_18['cyl'] = df_18['cyl'].astype(int)

The air_pollution_score, mpg columns, and greenhouse gas scores have a problem. According to [this link](http://www.fueleconomy.gov/feg/findacarhelp.shtml#airPollutionScore) (via the PDF documentation):

    "If a vehicle can operate on more than one type of fuel, an estimate is provided for each fuel type."
    
Vehicles with more than one fuel type will have a string that holds two values - one for each. 

In [164]:
def split_rows(df, column_list):
    
    #hybrids
    hb = df[df['fuel'].str.contains('/')]
    
    copy1 = hb.copy()
    copy2 = hb.copy() 

    for c in column_list:
        copy1[c] = copy1[c].apply(lambda x: x.split('/')[0])
        copy2[c] = copy2[c].apply(lambda x: x.split('/')[1])
        
    new_rows = copy1.append(copy2)
    
    df.drop(hb.index, inplace=True)
    df = df.append(new_rows, ignore_index=True)
    
    print("Number of rows split:", hb.shape[0])
    return df

In [165]:
df_08 = df_08.query('cert_region == "CA"')
df_18 = df_18.query('cert_region == "CA"')

df_08.drop('cert_region', axis=1,inplace=True)
df_18.drop('cert_region', axis=1,inplace=True)

#runs the above function for both dataframes

column_list = ['fuel','city_mpg', 'hwy_mpg', 'cmb_mpg', 'greenhouse_gas_score', 'air_pollution_score']
df_08 = split_rows(df_08, column_list)

df_18 = split_rows(df_18, column_list[:4])

Number of rows split: 1
Number of rows split: 76


In [166]:
for c in column_list[1:]:
    df_08[c] = df_08[c].astype(float)
    df_18[c] = df_18[c].astype(float)
    if c == 'greenhouse_gas_score':
        df_08[c] = df_08[c].astype(int)
        df_18[c] = df_18[c].astype(int)
        
df_08.dtypes == df_18.dtypes

model                   True
displ                   True
cyl                     True
trans                   True
drive                   True
fuel                    True
veh_class               True
air_pollution_score     True
city_mpg                True
hwy_mpg                 True
cmb_mpg                 True
greenhouse_gas_score    True
smartway                True
dtype: bool